In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
credits=pd.read_csv('/content/tmdb_5000_credits.csv')
movies = pd.read_csv('/content/tmdb_5000_movies.csv')

In [ ]:
credits.shape

(4803, 4)

In [ ]:
movies=movies.merge(credits , on='title')

In [ ]:
movies.shape

(4809, 23)

In [ ]:
movies=movies[['movie_id', 'title' , 'overview','genres','keywords','cast','crew']]

In [ ]:
movies.dropna(inplace=True)

In [ ]:
import ast 

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies['keywords'].apply(convert)

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4806, dtype: object

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
# what we want is the top 3 actors 

def convertCast(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
            
    return L

In [ ]:
movies['cast']=movies['cast'].apply(convertCast)

In [ ]:
def crewDirector(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=1:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['crew'].apply(crewDirector)

0       [Stephen E. Rivkin]
1          [Dariusz Wolski]
2           [Thomas Newman]
3             [Hans Zimmer]
4          [Andrew Stanton]
               ...         
4804     [Robert Rodriguez]
4805         [Edward Burns]
4806        [Carla Hetland]
4807          [Daniel Hsia]
4808       [Clark Peterson]
Name: crew, Length: 4806, dtype: object

In [ ]:
def getDirector(obj):
    L=[]
    for i in ast.literal_eval(obj):  
        if i['job']=='Director': 
            L.append(i['name'])
            break
   
    return L

In [ ]:
movies['crew']=movies['crew'].apply(getDirector)

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
# but here there is a problem , we will need to eliminate the spaces for eg
# sam mendes ==> sammendes as the model might get confused if there is 
# another sam and might recommend other sam 
# hence i will remove the spaces 

movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

0       [Action, Adventure, Fantasy, ScienceFiction]
1                       [Adventure, Fantasy, Action]
2                         [Action, Adventure, Crime]
3                   [Action, Crime, Drama, Thriller]
4                [Action, Adventure, ScienceFiction]
                            ...                     
4804                       [Action, Crime, Thriller]
4805                               [Comedy, Romance]
4806               [Comedy, Drama, Romance, TVMovie]
4807                                              []
4808                                   [Documentary]
Name: genres, Length: 4806, dtype: object

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
# similarly the above steps will be applied to other cols also 

movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
data=movies[['movie_id' , 'title' , 'tags']]

In [ ]:
data['tags']=data['tags'].apply(lambda x:" ".join(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data['tags']=data['tags'].apply(lambda x:x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=5000,stop_words='english')

In [ ]:
tfidf.fit_transform(data['tags']).toarray() # converting to an array as it should be done , no brainer , it is a vector na 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
vectors=tfidf.fit_transform(data['tags']).toarray()

In [ ]:
import nltk 
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
  List=[]
  for i in text.split():  # converting string into list 
    List.append(ps.stem(i))
  return " ".join(List)

In [ ]:
data['tags'].apply(stem) ## these stemminmg steps were supposed to be done before importing TFIDF 

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tags, Length: 4806, dtype: object

In [ ]:
data['tags']=data['tags'].apply(stem)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [ ]:
tfidf.get_feature_names() # these are all those 5000 words 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '17th',
 '18',
 '18th',
 '18thcenturi',
 '19',
 '1910',
 '1920',
 '1930',
 '1940',
 '1944',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1970',
 '1970s',
 '1971',
 '1974',
 '1976',
 '1980',
 '1985',
 '1990',
 '1999',
 '19th',
 '19thcenturi',
 '20',
 '200',
 '2003',
 '2009',
 '20th',
 '21st',
 '23',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '70',
 '80',
 'aaron',
 'aaroneckhart',
 'abandon',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'abov',
 'abus',
 'academ',
 'academi',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'account',
 'accus',
 'ace',
 'achiev',
 'acquaint',
 'act',
 'action',
 'actionhero',
 'activ',
 'activist',
 'activities',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adamsandl',
 'adamshankman',
 'adapt',
 'add',
 'addict',
 'adjust',
 'admir',
 'admit',
 'adolesc',
 'adopt',
 'ador',
 'adrienbrodi',
 'adult'

In [ ]:
# as the nunber of dimensions increase  the eucliedian distance tends to perform even worse ,  so we will be using cosine angle 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors)

array([[1.        , 0.02203984, 0.03038546, ..., 0.02319866, 0.        ,
        0.        ],
       [0.02203984, 1.        , 0.0129118 , ..., 0.01700865, 0.        ,
        0.00642518],
       [0.03038546, 0.0129118 , 1.        , ..., 0.01628553, 0.        ,
        0.        ],
       ...,
       [0.02319866, 0.01700865, 0.01628553, ..., 1.        , 0.01860761,
        0.02850017],
       [0.        , 0.        , 0.        , ..., 0.01860761, 1.        ,
        0.0197017 ],
       [0.        , 0.00642518, 0.        , ..., 0.02850017, 0.0197017 ,
        1.        ]])

In [ ]:
cosine_similarity(vectors).shape

(4806, 4806)

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
data[data['title']=='Batman Begins'].index[0]

119

In [ ]:
def recommendation(movie):
  movie_index=data[data['title']==movie].index[0] # this will return the index of the movie 
  distances=similarity[movie_index] # this will find the cosine similarity 
  movies_list=sorted(list(enumerate(distances)) , reverse=True , key = lambda x:x[1])[1:6]

  for i in movies_list:
    print(data.iloc[i[0]].title) # i eavh on eadi tahw i ma gniod 

In [ ]:
recommendation('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman v Superman: Dawn of Justice
Batman Returns


In [ ]:
recommendation('Avatar')

Aliens
Falcon Rising
Battle: Los Angeles
Aliens vs Predator: Requiem
Apollo 18


In [ ]:
data.iloc[0]['title'] # for the recommendation function i created 

'Avatar'

In [ ]:
recommendation("Hotel Transylvania 2")

Hotel Transylvania
The Beyond
The Best Exotic Marigold Hotel
Dracula Untold
Bobby


In [ ]:
data['title'].sample(10)

464                    Hotel Transylvania 2
559                            The Majestic
425                     Mission: Impossible
3427                       The Four Seasons
3192                               Hamlet 2
100     The Curious Case of Benjamin Button
1081                          The Ant Bully
3189                        The Ice Pirates
1365                                 Batman
354         The Girl with the Dragon Tattoo
Name: title, dtype: object

In [ ]:
recommendation('Iron Man 2')

Iron Man 3
Iron Man
Avengers: Age of Ultron
Captain America: Civil War
Ant-Man
